In [17]:
from IPython.display import HTML, display
from copy import deepcopy
import random
import os
import openai
import time
import sys
import ast
import json

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from getpass import getpass
import os
import pandas as pd

# import pdfkit
import html
from copy import deepcopy
import re

import re
from tqdm.auto import tqdm 
from datasets import load_dataset

In [18]:
# setting up an OpenAI template on the run
OPENAI_API_KEY = getpass()

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

### Setup Model

In [19]:
model = ChatOpenAI(model="gpt-4-0613")
model.temperature = 0.8

### HH more tasks

In [25]:
import re
from tqdm.auto import tqdm 
from datasets import load_dataset
raw_dataset = load_dataset("THUDM/AgentInstruct", split="alfworld")

task_list = []
for turn in raw_dataset:
    task_list.append(turn['conversations'][2]['value'].split("Here is your task. ")[-1])

Found cached dataset parquet (C:/Users/harsh/.cache/huggingface/datasets/THUDM___parquet/THUDM--AgentInstruct-ec062f298171fe56/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [28]:
system_prompt = \
"""You are given 5 examples of a task an agent can perform in an household environment. You have to make 3 more new and comprehensive tasks.
NOTE: Be diverse and realistic while innovating the tasks."""

more_task =[]
for _ in tqdm(range(100)):
    sample_tasks = random.sample(task_list, 5)
    user_message_str = ""
    for i, (t) in enumerate(sample_tasks):
        user_message_str += f"Task{i+1}: {t.strip()}\n\n"
    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=user_message_str.strip()),]
    response = model.predict_messages(messages)
    
    print(response.content)
    print('-'*50)
    
    new_tasks = [re.sub(r"Task\d+: ", "", t) for t in response.content.split("\n\n")]
    more_task.extend(new_tasks)

  0%|          | 0/100 [00:00<?, ?it/s]

Task6: You are in the middle of a room. Looking quickly around you, you see a dining table 1, a kitchen cabinet 1, a fridge 1, a garbage can 1, a stovetop 1, a sinkbasin 1, and a dishwasher 1.
Your task is to: find three dishes and put them in the dishwasher.

Task7: You are in the middle of a room. Looking quickly around you, you see a bed 1, a wardrobe 1, a nightstand 1, a dresser 1, a laundry hamper 1, a shelf 1, a mirror 1, and a carpet 1.
Your task is to: find and fold two shirts and put them in the dresser.

Task8: You are in the middle of a room. Looking quickly around you, you see a coffee table 1, a bookshelf 1, a sofa 1, a TV 1, a DVD player 1, a CD rack 1, a rug 1, and a lamp 1.
Your task is to: find a DVD, put it in the DVD player and turn on the TV.
--------------------------------------------------
Task6: You are in the middle of a room. Looking quickly around you, you see a cabinet 2, a couch 1, a diningtable 1, a drawer 1, a fridge 1, a garbagecan 1, a microwave 1, a sh

In [24]:
import pickle

with open('alfworld-more_task_list.pkl', 'wb') as file:
    pickle.dump(more_task, file)

NameError: name 'more_task' is not defined

In [4]:
import pickle
with open('alfworld-more_task_list.pkl', 'rb') as file:
    more_task = pickle.load(file)

### Multi-shot Example

In [28]:
example_1 = [ { "from": "human",  "value": "Interact with a household to solve a task. Imagine you are an intelligent agent in a household environment and your target is to perform actions to complete the task goal. At the beginning of your interactions, you will be given the detailed description of the current environment and your goal to accomplish. For each of your turn, you will be given a list of actions which you can choose one to perform in this turn. You should choose from two actions: \"THOUGHT\" or \"ACTION\". If you choose \"THOUGHT\", you should first think about the current condition and plan for your future actions, and then output your action in this turn. Your output must strictly follow this format:\"THOUGHT: your thoughts.\n ACTION: your next action\n\"; If you choose \"ACTION\", you should directly output the action in this turn. Your output must strictly follow this format:\"ACTION: your next action\n\". After your each turn, the environment will give you immediate feedback based on which you plan your next few steps. if the envrionment output \"Nothing happened\", that means the previous action is invalid and you should try more options.\n Reminder: \n1. the action must be chosen from the given available actions. Any actions except provided available actions will be regarded as illegal. \n2. Think when necessary, try to act directly more in the process." }, { "from": "gpt", "value": "OK. I'll follow your instructions and try my best to solve the task." }, { "from": "human",  "value": "Here is your task. You are in the middle of a room. Looking quickly around you, you see a bed 1, a desk 1, a drawer 8, a drawer 7, a drawer 6, a drawer 5, a drawer 4, a drawer 3, a drawer 2, a drawer 1, a dresser 1, a garbagecan 1, a shelf 5, a shelf 4, a shelf 3, a shelf 2, and a shelf 1.\nYour task is to: put two cellphone in dresser." }, { "from": "gpt",  "value": "THOUGHT: First, I need to find the cellphones. I'm going to start by looking in the most likely locations, such as the desk and the drawers.\n ACTION: go to desk 1" }, { "from": "human",  "value": "On the desk 1, you see a book 1, a bowl 1, a keychain 1, a laptop 1, and a pencil 2." }, { "from": "gpt",  "value": "ACTION: go to drawer 1" }, { "from": "human",  "value": "The drawer 1 is closed." }, { "from": "gpt",  "value": "ACTION: open drawer 1" }, { "from": "human",  "value": "You open the drawer 1. The drawer 1 is open. In it, you see nothing." }, { "from": "gpt",  "value": "ACTION: go to drawer 2" }, { "from": "human",  "value": "The drawer 2 is closed." }, { "from": "gpt",  "value": "ACTION: open drawer 2" }, { "from": "human",  "value": "You open the drawer 2. The drawer 2 is open. In it, you see nothing." }, { "from": "gpt",  "value": "ACTION: go to drawer 3" }, { "from": "human",  "value": "The drawer 3 is closed." }, { "from": "gpt",  "value": "ACTION: open drawer 3" }, { "from": "human",  "value": "You open the drawer 3. The drawer 3 is open. In it, you see nothing." }, { "from": "gpt",  "value": "ACTION: go to drawer 4" }, { "from": "human",  "value": "The drawer 4 is closed." }, { "from": "gpt",  "value": "ACTION: open drawer 4" }, { "from": "human",  "value": "You open the drawer 4. The drawer 4 is open. In it, you see nothing." }, { "from": "gpt",  "value": "ACTION: go to drawer 5" }, { "from": "human",  "value": "The drawer 5 is closed." }, { "from": "gpt",  "value": "ACTION: open drawer 5" }, { "from": "human",  "value": "You open the drawer 5. The drawer 5 is open. In it, you see nothing." }, { "from": "gpt",  "value": "ACTION: go to drawer 6" }, { "from": "human",  "value": "The drawer 6 is closed." }, { "from": "gpt",  "value": "ACTION: open drawer 6" }, { "from": "human",  "value": "You open the drawer 6. The drawer 6 is open. In it, you see nothing." }, { "from": "gpt",  "value": "ACTION: go to drawer 7" }, { "from": "human",  "value": "On the drawer 7, you see nothing." }, { "from": "gpt",  "value": "ACTION: open drawer 7" }, { "from": "human",  "value": "On the drawer 7, you see nothing." }, { "from": "gpt",  "value": "ACTION: go to drawer 8" }, { "from": "human",  "value": "The drawer 8 is closed." }, { "from": "gpt",  "value": "ACTION: open drawer 8" }, { "from": "human",  "value": "You open the drawer 8. The drawer 8 is open. In it, you see a creditcard 3." }, { "from": "gpt",  "value": "ACTION: go to shelf 1" }, { "from": "human",  "value": "On the shelf 1, you see nothing." }, { "from": "gpt",  "value": "ACTION: go to shelf 2" }, { "from": "human",  "value": "On the shelf 2, you see a alarmclock 1, a cellphone 1, and a creditcard 1." }, { "from": "gpt",  "value": "ACTION: take cellphone 1 from shelf 2" }, { "from": "human",  "value": "You pick up the cellphone 1 from the shelf 2." }, { "from": "gpt",  "value": "ACTION: go to shelf 3" }, { "from": "human",  "value": "On the shelf 3, you see a pencil 1." }, { "from": "gpt",  "value": "ACTION: go to shelf 4" }, { "from": "human",  "value": "On the shelf 4, you see a cd 1." }, { "from": "gpt",  "value": "ACTION: go to shelf 5" }, { "from": "human",  "value": "On the shelf 5, you see nothing." }, { "from": "gpt",  "value": "ACTION: go to dresser 1" }, { "from": "human",  "value": "On the dresser 1, you see a alarmclock 2, a cd 2, a creditcard 2, a desklamp 1, a pen 2, and a pen 1." }, { "from": "gpt",  "value": "ACTION: put cellphone 1 in/on dresser 1" }, { "from": "human",  "value": "You put the cellphone 1 in/on the dresser 1." }, { "from": "gpt",  "value": "THOUGHT: I still need another cellphone to complete the task. The desk and the shelf could be likely places where I can find one.\n ACTION: go to desk 1" }, { "from": "human",  "value": "On the desk 1, you see a book 1, a bowl 1, a keychain 1, a laptop 1, and a pencil 2." }, { "from": "gpt",  "value": "ACTION: go to shelf 2" }, { "from": "human",  "value": "On the shelf 2, you see a alarmclock 1, and a creditcard 1." }, { "from": "gpt",  "value": "ACTION: go to shelf 3" }, { "from": "human",  "value": "On the shelf 3, you see a pencil 1." }, { "from": "gpt",  "value": "ACTION: go to shelf 4" }, { "from": "human",  "value": "On the shelf 4, you see a cd 1." }, { "from": "gpt",  "value": "ACTION: go to shelf 5" }, { "from": "human",  "value": "On the shelf 5, you see nothing." }, { "from": "gpt",  "value": "ACTION: go to bed 1" }, { "from": "human",  "value": "On the bed 1, you see a book 2, a cellphone 3, a cellphone 2, and a pillow 1." }, { "from": "gpt",  "value": "ACTION: take cellphone 3 from bed 1" }, { "from": "human",  "value": "You pick up the cellphone 3 from the bed 1." }, { "from": "gpt",  "value": "ACTION: go to dresser 1" }, { "from": "human",  "value": "On the dresser 1, you see a alarmclock 2, a cd 2, a cellphone 1, a creditcard 2, a desklamp 1, a pen 2, and a pen 1." }, { "from": "gpt",  "value": "ACTION: put cellphone 3 in/on dresser 1" } ]

In [29]:
one_shot_messages = []
for dialog in example_1:
    if dialog['from'] == 'human':
        msg = HumanMessage(content=dialog['value'])
    elif dialog['from'] == 'gpt':
        msg = AIMessage(content=dialog['value'])
    one_shot_messages.append(msg)
    

In [30]:
one_shot_messages

[HumanMessage(content='Interact with a household to solve a task. Imagine you are an intelligent agent in a household environment and your target is to perform actions to complete the task goal. At the beginning of your interactions, you will be given the detailed description of the current environment and your goal to accomplish. For each of your turn, you will be given a list of actions which you can choose one to perform in this turn. You should choose from two actions: "THOUGHT" or "ACTION". If you choose "THOUGHT", you should first think about the current condition and plan for your future actions, and then output your action in this turn. Your output must strictly follow this format:"THOUGHT: your thoughts.\n ACTION: your next action\n"; If you choose "ACTION", you should directly output the action in this turn. Your output must strictly follow this format:"ACTION: your next action\n". After your each turn, the environment will give you immediate feedback based on which you plan 

In [32]:
for g in one_shot_messages:
    print(g.content)
    print("")

Interact with a household to solve a task. Imagine you are an intelligent agent in a household environment and your target is to perform actions to complete the task goal. At the beginning of your interactions, you will be given the detailed description of the current environment and your goal to accomplish. For each of your turn, you will be given a list of actions which you can choose one to perform in this turn. You should choose from two actions: "THOUGHT" or "ACTION". If you choose "THOUGHT", you should first think about the current condition and plan for your future actions, and then output your action in this turn. Your output must strictly follow this format:"THOUGHT: your thoughts.
 ACTION: your next action
"; If you choose "ACTION", you should directly output the action in this turn. Your output must strictly follow this format:"ACTION: your next action
". After your each turn, the environment will give you immediate feedback based on which you plan your next few steps. if th

In [35]:
example_1[3]

{'from': 'gpt',
 'value': "THOUGHT: First, I need to find the cellphones. I'm going to start by looking in the most likely locations, such as the desk and the drawers.\n ACTION: go to desk 1"}

### Householding Environment

In [21]:
environment_prompt_template = '''Your Goal is to simulate a household environment that an agent is acting in. You will provide the output from the environment.  Below is the task the agent has been given. 

Task: 

{}

--------
For each turn, you will receive an action chosen by the agent. You must then respond as the environment listing any available actions that the agent may take next. If the user does not select an action from the provided actions, you must reply "Nothing happened." You should strive to provide actions that are realistic to the environment. You should always remind the agent of any items they possess. If the user has accomplished the task, YOU MUST call the success function: "success('[why task was successful]')". You should make sure to provide actions that relate to any items the agent possess and permit the agent to complete the goal. YOU MUST PROVIDE ACTIONS THAT ARE CONSISTENT WITH YOUR PAST ACTIONS. Please acknowledge that you understand and let's begin.

'''

hh_agent_prompt =  "Interact with a household to solve a task. Imagine you are an intelligent agent in a household environment and your target is to perform actions to complete the task goal. At the beginning of your interactions, you will be given the detailed description of the current environment and your goal to accomplish. For each of your turn, you will be given a list of actions which you can choose one to perform in this turn. You should choose from two actions: \"THOUGHT\" or \"ACTION\". If you choose \"THOUGHT\", you should first think about the current condition and plan for your future actions, and then output your action in this turn. Your output must strictly follow this format:\"THOUGHT: your thoughts.\n ACTION: your next action\n\"; If you choose \"ACTION\", you should directly output the action in this turn. Your output must strictly follow this format:\"ACTION: your next action\n\". After your each turn, the environment will give you immediate feedback based on which you plan your next few steps. if the envrionment output \"Nothing happened\", that means the previous action is invalid and you should try more options.\n Reminder: \n1. the action must be chosen from the given available actions. Any actions except provided available actions will be regarded as illegal. \n2. Think when necessary, try to act directly more in the process."

hh_agent_response = "OK. I'll follow your instructions and try my best to solve the task."



In [9]:
task = "Here is your task. You are in the middle of a room. Looking quickly around you, you see a armchair 2, a armchair 1, a bed 1, a countertop 1, a diningtable 1, a drawer 2, a drawer 1, a dresser 1, a garbagecan 1, a laundryhamper 1, and a sidetable 1.\nYour task is to: find two laptop and put them in bed."


In [38]:
agent_messages = one_shot_messages + [
    HumanMessage(content=hh_agent_prompt),
    AIMessage(content=hh_agent_response),
    HumanMessage(content=task)
]




In [39]:
agent_messages

[HumanMessage(content='Interact with a household to solve a task. Imagine you are an intelligent agent in a household environment and your target is to perform actions to complete the task goal. At the beginning of your interactions, you will be given the detailed description of the current environment and your goal to accomplish. For each of your turn, you will be given a list of actions which you can choose one to perform in this turn. You should choose from two actions: "THOUGHT" or "ACTION". If you choose "THOUGHT", you should first think about the current condition and plan for your future actions, and then output your action in this turn. Your output must strictly follow this format:"THOUGHT: your thoughts.\n ACTION: your next action\n"; If you choose "ACTION", you should directly output the action in this turn. Your output must strictly follow this format:"ACTION: your next action\n". After your each turn, the environment will give you immediate feedback based on which you plan 

In [40]:
agent_response = model.predict_messages(agent_messages)

In [41]:
agent_response.content

"THOUGHT: I'll start by looking for laptops on the most likely places such as the dining table, the countertop, and the drawer.\n ACTION: go to diningtable 1"

In [17]:
# Create one shot prompt

In [27]:

one_shot_environment_prompt = environment_prompt_template.format(example_1[2]['value'])

one_shot_environment_messages = [
    HumanMessage(content=one_shot_environment_prompt),
    AIMessage(content="OK. I'll follow your instructions and try my best to simulate a household environment."),
]

for dialog in example_1[3:]:
    if dialog['from'] == 'human':
        msg = AIMessage(content=dialog['value'])
    elif dialog['from'] == 'gpt':
        msg = HumanMessage(content=dialog['value'])
    one_shot_environment_messages.append(msg)

one_shot_environment_messages.append(AIMessage(content="success('cellphone 1 and cellphone 3 were placed on dresser 1')"))

NameError: name 'example_1' is not defined

In [43]:
environment_prompt = environment_prompt_template.format(task)

environment_messages = one_shot_environment_messages + [
    HumanMessage(content=environment_prompt),
    AIMessage(content="OK. I'll follow your instructions and try my best to simulate a household environment."),
    HumanMessage(content=agent_response.content)
]

In [44]:
environment_result = model.predict_messages(environment_messages)

In [45]:
environment_result.content

'On the diningtable 1, you see a book 1, a bowl 1, a keychain 1, a laptop 1, and a pen 1.'

In [46]:
environment_messages

[HumanMessage(content='Your Goal is to simulate a household environment that an agent is acting in. You will provide the output from the environment.  Below is the task the agent has been given. \n\nTask: \n\nHere is your task. You are in the middle of a room. Looking quickly around you, you see a bed 1, a desk 1, a drawer 8, a drawer 7, a drawer 6, a drawer 5, a drawer 4, a drawer 3, a drawer 2, a drawer 1, a dresser 1, a garbagecan 1, a shelf 5, a shelf 4, a shelf 3, a shelf 2, and a shelf 1.\nYour task is to: put two cellphone in dresser.\n\n--------\nFor each turn, you will receive an action chosen by the agent. You must then respond as the environment listing any available actions that the agent may take next. If the user does not select an action from the provided actions, you must reply "Nothing happened." You should strive to provide actions that are realistic to the environment. You should always remind the agent of any items they posess. If the user has accomplished the task,

In [47]:
agent_messages.append(HumanMessage(content=environment_result.content))

In [48]:
num_turns = 10

for i in range(num_turns):
    agent_response = model.predict_messages(agent_messages)
    print(agent_response.content)
    agent_result = agent_response.content
    environment_messages.append(HumanMessage(content=agent_result))
    agent_messages.append(agent_response)
    environment_result = model.predict_messages(environment_messages)
    if "success(" in environment_result.content:
        break
    environment_messages.append(environment_result)
    print(environment_result.content)
    agent_messages.append(HumanMessage(content=environment_result.content))

ACTION: take laptop 1 from diningtable 1
Nothing happened.
THOUGHT: It seems there was an error with my previous action. I should try again.
 ACTION: go to diningtable 1
On the diningtable 1, you see a cup 1, a fork 1, a knife 1, a laptop 1, and a plate 1.
ACTION: take laptop 1 from diningtable 1
You pick up the laptop 1 from the diningtable 1.
ACTION: go to bed 1
On the bed 1, you see a pillow 1.
ACTION: put laptop 1 on bed 1
You put the laptop 1 on the bed 1.
THOUGHT: Now I need to find the second laptop. I'll check the other places around the room.
 ACTION: go to drawer 1
The drawer 1 is closed.
ACTION: open drawer 1
You open the drawer 1. The drawer 1 is open. In it, you see a notepad 1 and a pencil 1.
ACTION: go to drawer 2
The drawer 2 is closed.
ACTION: open drawer 2
You open the drawer 2. The drawer 2 is open. In it, you see a remotecontrol 1.
ACTION: go to countertop 1
On the countertop 1, you see a glass 1, a jug 1, and a laptop 2.


#### Get tasks from AgentInstruct Data

In [49]:
import pandas as pd
df = pd.read_parquet('~/spikes/notebook_spikes/agentinstruct/alfworld-00000-of-00001-302ad687bb3817a4.parquet')
print(df)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\harsh/spikes/notebook_spikes/agentinstruct/alfworld-00000-of-00001-302ad687bb3817a4.parquet'

In [22]:
from datasets import load_dataset
raw_dataset = load_dataset("THUDM/AgentInstruct", split="alfworld")
dataset = []
for ds in raw_dataset:
    dataset.append(ds)
df = pd.DataFrame(dataset)

Found cached dataset parquet (C:/Users/harsh/.cache/huggingface/datasets/THUDM___parquet/THUDM--AgentInstruct-ec062f298171fe56/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [52]:
raw_dataset[0]['conversations']

[{'from': 'human',
  'loss': None,
  'value': 'Interact with a household to solve a task. Imagine you are an intelligent agent in a household environment and your target is to perform actions to complete the task goal. At the beginning of your interactions, you will be given the detailed description of the current environment and your goal to accomplish. For each of your turn, you will be given a list of actions which you can choose one to perform in this turn. You should choose from two actions: "THOUGHT" or "ACTION". If you choose "THOUGHT", you should first think about the current condition and plan for your future actions, and then output your action in this turn. Your output must strictly follow this format:"THOUGHT: your thoughts.\n ACTION: your next action\n"; If you choose "ACTION", you should directly output the action in this turn. Your output must strictly follow this format:"ACTION: your next action\n". After your each turn, the environment will give you immediate feedback 

In [53]:
print(df.iloc[2]["conversations"][2]['value'])

Here is your task. You are in the middle of a room. Looking quickly around you, you see a cabinet 11, a cabinet 10, a cabinet 9, a cabinet 8, a cabinet 7, a cabinet 6, a cabinet 5, a cabinet 4, a cabinet 3, a cabinet 2, a cabinet 1, a coffeemachine 1, a countertop 2, a countertop 1, a diningtable 2, a diningtable 1, a drawer 3, a drawer 2, a drawer 1, a fridge 1, a garbagecan 1, a microwave 1, a sidetable 1, a sinkbasin 1, a stoveburner 4, a stoveburner 3, a stoveburner 2, a stoveburner 1, and a toaster 1.
Your task is to: heat some plate and put it in countertop.


In [54]:
def process(data):
    return (data['conversations'][2]['value'])

In [55]:
task_and_environment = df.apply(process, axis=1).tolist()

In [56]:
len(task_and_environment )

336

### Create Training Data Conversations

In [10]:
def one_shot_environment_setup():
    one_shot_environment_prompt = environment_prompt_template.format(example_1[2]['value'])
    one_shot_environment_messages = [
        HumanMessage(content=one_shot_environment_prompt),
        AIMessage(content="OK. I'll follow your instructions and try my best to simulate a household environment."),
    ]

    for dialog in example_1[3:]:
        if dialog['from'] == 'human':
            msg = AIMessage(content=dialog['value'])
        elif dialog['from'] == 'gpt':
            msg = HumanMessage(content=dialog['value'])
        one_shot_environment_messages.append(msg)

    one_shot_environment_messages.append(AIMessage(content="success('cellphone 1 and cellphone 3 were placed on dresser 1')"))
    return one_shot_environment_messages

In [26]:
one_shot_environment_messages

NameError: name 'one_shot_environment_messages' is not defined

In [23]:
num_turns = 40

all_agents = []
all_environments = []
start = 0

for x in tqdm(range(start,start+len(more_task))): 
    task = more_task[x]
    print("Task {}:".format(x), task)
    agent_messages = one_shot_messages + [
        # HumanMessage(content=hh_agent_prompt),
        # AIMessage(content=hh_agent_response),
        HumanMessage(content=f"Here is your task. {task}")
    ]

    agent_response = model.predict_messages(agent_messages)
    print(agent_response.content)
    agent_messages.append(agent_response)
    
    # environment_message = one_shot_environment_setup()
    environment_prompt = environment_prompt_template.format(task)
    environment_messages = one_shot_environment_messages + [
        HumanMessage(content=environment_prompt),
        AIMessage(content="OK. I'll follow your instructions and try my best to simulate a household environment."),
        HumanMessage(content=agent_response.content)
    ]
    environment_result = model.predict_messages(environment_messages)
    environment_messages.append(environment_result)
    agent_messages.append(HumanMessage(content=environment_result.content))
    print(environment_result.content)
    for i in range(num_turns):
        agent_response = model.predict_messages(agent_messages)
        print(agent_response.content)
        environment_messages.append(HumanMessage(content=agent_response.content))
        agent_messages.append(agent_response)
        environment_result = model.predict_messages(environment_messages)
        environment_messages.append(environment_result)
        print(environment_result.content)
        agent_messages.append(HumanMessage(content=environment_result.content))
        if "success(" in environment_result.content:
            break
    all_agents.append(deepcopy(agent_messages))
    all_environments.append(deepcopy(environment_messages))
    print("\n\n")

NameError: name 'more_task' is not defined

In [71]:
len(all_environments)

2

In [19]:
conversations_ = []
for chat in all_agents:
    conversation_ = []
    for side in chat:
        if side.type == "human":
            side_type = "human"
        else:
            side_type = "gpt"
        conversation_.append({"from": side_type, "value":side.content})
    conversations_.append(conversation_ )
        

In [20]:
with open('./synthetic-0/householding-interactions-raw-1.jsonl', 'w') as file:
    for conversation in conversations_:
        json_str = json.dumps(conversation)
        file.write(json_str + "\n")

### Few Shot

In [1]:
from langchain.chat_models import ChatOpenAI
from getpass import getpass
from langchain.schema import HumanMessage, AIMessage, SystemMessage
import os
import json

In [2]:
# setting up an OpenAI template on the run
OPENAI_API_KEY = getpass()

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [6]:
model = ChatOpenAI(model="gpt-4")
model.temperature = 0.8

In [3]:
from datasets import load_dataset
all_data = []
dataset = load_dataset("THUDM/AgentInstruct", split="alfworld")
for ds in dataset:
    all_data.append(ds)

Found cached dataset parquet (C:/Users/harsh/.cache/huggingface/datasets/THUDM___parquet/THUDM--AgentInstruct-ec062f298171fe56/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [4]:
def convert_to_convs(s):
    s = s.split("## USER: ")
    s = [ss for ss in s if ss.strip()!="" ]
    s = ["## USER: "+ss for ss in s]
    _dict = {"conversations": []}
    for ss in s:
        for sss in ss.split("## USER:")[1:]:
            user_conv, agent_conv = sss.split('## AGENT:')[0].strip(), sss.split('## AGENT:')[1].strip()
            _dict["conversations"] += [{"from": "human", "value": user_conv, "loss": False}, {"from": "gpt", "value": agent_conv, "loss": True}]
    return _dict

In [16]:
s = gen_conv
s = s.split("## USER: ")
s = [ss for ss in s if ss.strip()!=""]
s = ["## USER: "+ss for ss in s]

_dict = {"conversations": []}
for ss in s:
    for sss in ss.split("## USER:")[1:]:
        print(sss)
        print('------')
        # user_conv, agent_conv = sss.split('## AGENT:')[0].strip(), sss.split('## AGENT:')[1].strip()
        # _dict["conversations"] += [{"from": "human", "value": user_conv, "loss": False}, {"from": "gpt", "value": agent_conv, "loss": True}]

 Here is your task. You are in the middle of a room. Looking quickly around you, you see a cabinet 10, a cabinet 9, a cabinet 8, a cabinet 7, a cabinet 6, a cabinet 5, a cabinet 4, a cabinet 3, a cabinet 2, a cabinet 1, a coffeemachine 1, a countertop 2, a countertop 1, a diningtable 1, a drawer 5, a drawer 4, a drawer 3, a drawer 2, a drawer 1, a fridge 1, a garbagecan 1, a microwave 1, a sinkbasin 1, a stoveburner 4, a stoveburner 3, a stoveburner 2, a stoveburner 1, and a toaster 1.
Your task is to: clean the countertop 1.

## AGENT: THOUGHT: The task is to clean the countertop 1. I need to find a cleaning tool or a cloth, then use it to clean the countertop 1. The cleaning tool could be in the kitchen cabinets or drawer. Let's look around.
 ACTION: look


------
 You are in the middle of a room. Looking quickly around you, you see a cabinet 10, a cabinet 9, a cabinet 8, a cabinet 7, a cabinet 6, a cabinet 5, a cabinet 4, a cabinet 3, a cabinet 2, a cabinet 1, a coffeemachine 1, a c

In [7]:
from tqdm.auto import tqdm 
import random
num_turns = 1000
init_prompt = \
"""Given is a conversations between an AI agent and a user. The user asks the agent to do a task in a household environment and in return the agent does it chains of thoughts and actions. The agent produces actions and the user returns the state of the environment. You have to generate some more conversations following the previous. Add more verbose in the conversations."""
init_task_gen_prompt = \
"""
Given are some tasks which can be performed in a household environment. Can you generate *one* more such task which is different from the previous given ones?
"""
all_convs = []
all_tasks = []
for _ in tqdm(range(num_turns)):
    random.shuffle(all_data)
    random.shuffle(all_tasks)
    prompt = ""
    _value = all_data[0]['conversations'][0]['value']
    prompt+=f"\n\n## USER: {_value}"
    _value = all_data[0]['conversations'][1]['value']
    prompt+=f"\n\n## AGENT: {_value}"
    for d in all_data[1:3]:
        for dd in d["conversations"][2:]:
            _from = "USER" if dd['from']=="human" else "AGENT"
            prompt += f"\n\n## {_from}: {dd['value']}"
        all_tasks.append(d["conversations"][2]['value'].replace("Here is your task. ", ""))
    task_messages =[
        SystemMessage(content=init_task_gen_prompt),
        HumanMessage(content="\n\n".join(all_tasks[-100:]))
    ]
    
    
    task_response = model.predict_messages(task_messages)
    task = task_response.content.strip()
    messages = [
        SystemMessage(content=init_prompt),
        HumanMessage(content=prompt+f"\n\n## USER: Here is your task. {task}")
    ]   
    response = model.predict_messages(messages)
    gen_conv = f"## USER: Here is your task. {task}\n\n" + response.content
    if len(gen_conv.split("## AGENT:"))>1:
        convs = convert_to_convs(gen_conv)
        all_convs.append(convs)    
        messages.append(response)

    task_response = model.predict_messages(task_messages)
    task = task_response.content.strip()
    messages = [
        SystemMessage(content=init_prompt),
        HumanMessage(content=prompt+f"\n\n## USER: Here is your task. {task}")
    ]   
    response = model.predict_messages(messages)
    gen_conv = f"## USER: Here is your task. {task}\n\n" + response.content
    if len(gen_conv.split("## AGENT:"))>1:
        convs = convert_to_convs(gen_conv)
        all_convs.append(convs)    
        messages.append(response)

    with open('./synthetic-3/hh_syn-datasyntype2-2.json', 'w') as file:
        json.dump(all_convs, file)
        # for d in all_convs:
        #     json.dump(d, file)
        #     file.write('\n')
    # break

  0%|          | 0/1000 [00:00<?, ?it/s]

IndexError: list index out of range